# 6. Multi-Branch Networks (GoogLeNet)

In [2]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

import numpy as np
import pandas as pd

The key contribution in GoogLeNet was that it solved the problem of **selecting convolution kernels**.

While other works tried to identify the **optimal convolution kernel** with shape ranging from $1 \times 1$ to $11 \times 11$, GoogLeNet simply **concatenated multi-branch convolutions**.

In the following, we introduce a **slightly simplified version** of GoodLeNet: some tricks are no longer needed due to the improvedf learning algorihtms nowedays.

## Inception Blocks

The basic convolutional block in GoogLeNet is called an **`inception block`**:

![](http://d2l.ai/_images/inception.svg)

As shown above, an inception block consists of **4 parallel branches**. The first 3 branches use convolutional layers with window size $1 \times 1$, $3 \times 3$ and $5 \times 5$.\, trying to **extract information** from different spatial sizes. 

The middle two branches include a $1 \times 1$ convolutional layer to reduce the number of channels and **model complexity**. The fourth branch uses a **max-pooling layer** followed by a $1 \times 1$ convolutional layer to change the **number of channels**.

All branches uses appropriate **padding** to maintain the size of the input. Finally, the outputs of all branches are **concatenated** along the channel dimension.

The commonly tuned **hyperparameters** of the inception block are the **number of output channels per layer**.

In [3]:
class Inception(nn.Module):

    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        #path 1: 1×1 convolutional layer
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        
        #path 2: 1×1 convolutional layer + 3×3 convolution layer
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        
        #path 3: 1×1 convolutional layer + 5×35 convolution layer
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        
        #path 4: 3×3 max-pooling layer + 1×1 convolutional layer
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)

## GoogLeNet Model

As shown below, **GoogLeNet** consists of **9 inception blocks** with **max-pooling** layers in between (to reduce dimensionality), and ends up with a **global average pooling** layer to generate outputs.

![](http://d2l.ai/_images/inception-full-90.svg)


In [5]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [7]:
X = torch.rand(size=(1, 1, 96, 96))

for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])
